# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Импортируем библиотеку pandas и сохраним данные в переменной `df`

In [1]:
import pandas as pd # импорт библиотеки pandas 

df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохраbнение в df

Выведем общую информацию о данных:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно исходным данным, в таблице содержатся следующие названия столбцов:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названии столбцов нет нарушений общего стиля, все названия понятны и просты.

Выведем первые 10 значений таблицы\: 

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В каждой строке содержится информация о каждом заёмщике. 

В первых строках видно, что некоторые данные содержат значения с верхним регистром и отрицательные значения. А так же значения, которые являются возможными артефактами: например в столбце о количестве дней стажа - `340266.072047`.

В столбце `education` есть одинаковые значения, но с разным регистром, что нужно исправить.

Каждое значение столбца `education` соответствует уникальному значению в столбце `education_id`, так же как и в столбцах `family_status` и `family_status_id`.

В столбцах `days_employed` и `total_income` таблицы содержатся пропуски, которые, возможно, связаны между собой. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Выявим количество пропусков в таблице и выведем те строки, в которых пропущены значения.

In [4]:
df.isna().sum() # вывод количества пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
df[df['days_employed'].isna()].head(10) # вывод строк, в которых есть пропуски

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Видно, что пропущенные значения в столбцах `days_employed` и `total_income` связаны между собой, но не связаны с остальными значениями других строк. Возможно, что заёмщик не указывал свой рабочий стаж и ежемесячный доход.

В изначальной задаче нет цели, чтобы проанализировать влияние трудового стажа и дохода на факт погащения кредита в срок, но эти строки данных необходимы в дальнейшем. Соответсвенно есть несколько решений:

* заполнить пропущенние значения в таблице нулями и продолжить обработку данных.
* заполнить значения в столбце `days_employed` средним значением по всему столбцу и в столбце `total_income` - медианным значением.
* заполнить значения в столбцах, как указано выше, но с учетом типа занятости в столбце `income_type` и образованием.

Строки с значением 0 могут повлиять на исследование, а замена на средние и медианные значения по всему столбцу, могут исказить исследования, поэтому воспользуемся третьим вариантом.

Выявим значения в столбце `total_income` по типу занятости `income_type` медианным значением:

In [6]:
df['total_income'] = df['total_income']\
                    .fillna(df.groupby(['income_type', 'education_id'])\
                    ['total_income'].transform('median'))

Прежде, чем заполнять данные в столбце `days_employed` необходимо выявить артефакты и как они взаимосвязаны между собой.

По некоторым строкам столбца видно, что отрицательные значения связаны с столбцом `income_type`, то есть с занятостью заёмщика.

Проверим гипотезу:

In [7]:
df[df['days_employed'] <= 0]['income_type'].unique() # выводим уникальные значения столбца при отрицаьтельных значениях

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

In [8]:
df[df['days_employed'] >= 0]['income_type'].unique() # выводим уникальные значения столбца при положительных значениях

array(['пенсионер', 'безработный'], dtype=object)

Гипотеза подтвердилась, соотвественно, необходимо обработать данные в столбце и заменить отрицательные значения на положительные.

Заменим отрицательные значения на положительные, умножив их на -1:

In [9]:
df['days_employed'] = df['days_employed'].apply(abs) # заменяем значения

df[df['days_employed'] >= 0] # выведем информацию о таблице

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Отрицательных значений в таблице нет.

Далее необходимо разобраться в значениях-артефактах.

Вероятно, при указании занятости, значения в столбце стажа работы указывают на то, сколько дней человек пребывает в качестве этого типа занятости. Возможно, при заполнении стажа работы при занятости "пенсионер" и "безработный" запятая в значении была сдвинута или число было умножено на 100.

Исправим данный артефакт в таблице, разделив эти значения на 100:

In [10]:
def change_days(row): # создаём функцию для строки, которая делит значение в стаже на 100
    days = row['days_employed']
    income = row['income_type']
    if income == 'безработный' or income == 'пенсионер':
        days = days / 100
    return days

df['days_employed'] = df.apply(change_days, axis=1) # применяем функцию

Артефакты в столбце откорректированы, теперь заполним пропущенные значения в стаже на среднее значение:

In [11]:
days_employed_mean = df.groupby('income_type')['days_employed'].mean() #выделяем средние значения по типу занятости

def fill_days(row): # создаем функцию по замене значений
    if pd.isna(row['days_employed']):
        return days_employed_mean.loc[row['income_type']]
    return row['days_employed']
 
df['days_employed'] = df.apply(fill_days, axis=1) # заполняем данные средним значением

Проверим, остались ли в таблице пропуски:

In [12]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски в таблице заполнены, теперь можно двигаться дальше.

### Замена типа данных и исправление ошибок

Для предотвращения возможных ошибок, необходимо изменить типы вещественных данных на целочисленные.

Заменим вещественных на целочисленные в столбцах `days_employed` и `total_income`:

In [13]:
# замена на целочисленные значения с помощью astype

df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

В столбце `education` есть значения, которые отличаются по регистру, исправим эту неточность:

In [14]:
# замена регистра с помощью lower

df['education'] = df['education'].str.lower() 
df['gender'] = df['gender'].str.lower()

Значения в некоторых столбцах соотвестуют их уникальному идентификатору. Выделим отдельные словари для этих индентификаторов:

In [15]:
family_status_dict = df[['family_status', 'family_status_id']] # создаем таблицу

family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True) # удаляем дубликаты

family_status_dict # выводим на экран

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [16]:
education_dict = df[['education', 'education_id']] # создаем таблицу

education_dict = education_dict.drop_duplicates().reset_index(drop=True) # удаляем дубликаты

education_dict # выводим на экран

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Выявим уникальные значения в столбцах, необходимые для дальнейшей обработки данных:

Проверим уникальное количество детей:

In [17]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` содержится аномальное значение - `20` детей и `-1`. 
При заполнении данного столбца были допущены ошибки. 

Заменим значения в столбце на положительные и уменьшим количество `20` на `2`:

In [18]:
# замена с помощью replace

df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

Проверим возраст:

In [19]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

В данных есть значения `0`.

Чтобы не исказить значения по возрасту, заменим возраст `0` на средние значения возраста по типу занятости `income_type`:

In [20]:
dob_years_mean = df.groupby('income_type')['dob_years'].mean().astype(int) #выделяем средние значения по типу занятости

def fill_age(row): # создаем функцию по замене значений
    if row['dob_years'] == 0:
        return dob_years_mean.loc[row['income_type']]
    return row['dob_years']
 
df['dob_years'] = df.apply(fill_age, axis=1) # заполняем данные средним значением

**Вывод**

Типы данных заменены на целочисленные, значения с разным регистром в категорийных данных исправлены.

Выделены словари для идентификаторов в столбцах `education` и `family_status`:

Для образования:
* 0 - высшее
* 1 - среднее
* 2 - неоконченное высшее
* 3 - начальное
* 4 - ученая степень

Для семейного статуса:
* 0 - женат / замужем
* 1 - гражданский брак
* 2 - вдовец / вдова
* 3 - в разводе
* 4 - Не женат / не замужем

Ошибки в столбцах исправлены и более не повлияют на результат.

### Обработка дубликатов

Выявим количество дубликатов:

In [21]:
df.duplicated().sum() # подсчет количества дубликатов

71

В таблице содержится 71 дубликат.

Исключим дупликаты в таблице, для дальнейшей обработки данных:

In [22]:
df = df.drop_duplicates().reset_index(drop=True) # исключаем дубликаты

Проверим количетсво дубликатов:

In [23]:
df.duplicated().sum() # подсчет количества дубликатов

0

**Вывод**

Дубликаты из таблицы исключены, можно двигаться дальше.

### Лемматизация

Приведем слова в столбце `purpose` к его словарной форме **(лемме)** с помощью библиотеки `pymystem3`:

In [24]:
from pymystem3 import Mystem # импортируем библиотеку

m = Mystem() # привязка к переменной

def do_lemma(text): # создаем функцию для лемматизации текста
    lemma = m.lemmatize(text)
    return lemma

df['lemma_purpose'] = df['purpose'].apply(do_lemma) # лемматизируем текст и добавляем отдельный столбец с леммамии

Выведем результат:

In [25]:
df['lemma_purpose']

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: lemma_purpose, Length: 21454, dtype: object

**Вывод**

Данные в столбце успешно лемматизированы. 

Новый столбец можно использовать для категоризации данных по цели получения кредита.

### Категоризация данных

Для последующего анализа данных, категоризируем некоторые столбцы по их значениям и по цели нашего исследования:

* по количеству детей (для значений более 3 выведем отдельную категорию)
* по ежемесячному доходу
* по цели кредита

Семейное положение уже распределено по категориям, соотвественно, категоризация не требуется.

**Распределим по количеству детей.**

Добавим новый столбец `children_cat` и введем категорию `более 3` для клиента с 3 и более детьми.

In [26]:
def children_for_cat(children): # создаем функцию для возврата значения, если оно более 3
    if children >=3:
        return 'более 3'
    else:
        return children
    
df['children_cat'] = df['children'].apply(children_for_cat) # создаем новый столбец и вводим туда значения

df['children_cat'].value_counts() # подсчитаем количество значений

0          14091
1           4855
2           2128
более 3      380
Name: children_cat, dtype: int64

Данные по количеству детей теперь упрощены.

**Распределим по ежемесячному доходу.**

Воспользуемся встроенной функцией `pandas` - `qcut`.

`qcut` позволит разделить базовые данные на интервалы равного размера, что поможет в анализе.

Разделим на 5 категорий:

* низкий
* ниже среднего
* средний
* выше среднего
* высокий

In [27]:
total_income_categ = ['низкий', 'ниже среднего', 'средний', 'выше среднего', 'высокий'] # список с категориями

df['total_income_cat'] = pd.qcut(df['total_income'], # воспользуемся qcut для категоризации
                                 q=5, 
                                 labels=total_income_categ)

df['total_income_cat'].value_counts() # подсчитаем количество значений 

низкий           4291
ниже среднего    4291
выше среднего    4291
высокий          4291
средний          4290
Name: total_income_cat, dtype: int64

Данные по уровню дохода категоризированы по имеют равное количество, разделенное по категориям.

**Распределим по цели кредита.**

Выведем на экран уникальные значения `purpose`

In [28]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Как видно из уникальных значений, можно разделить цель кредита по нескольким явным категориям:

* операции с автомобилем
* операции с недвижимостью
* операции с жильем
* свадьба
* образование

Возможно, операции с недвижимостью и операции с жильем можно объединить в общую категорию `недвижимость` и проверить, как влияет на статистику с разделением и без. Проверим гипотезу уже после разделения на 5 категорий.

Воспользуемся ранее лемматизированными данными и распределим по категориям:

In [29]:
purpose_list = ['автомобиль', 'жилье', 'свадьба', 'образование', 'недвижимость'] # список с категориями

def purpose_short(purpose): # функция, которая возвращает название категории, если это слово содержится в списке лемм
    for text_purpose in purpose:
        for text_lemma in purpose_list:
            if text_purpose == text_lemma:
                return text_purpose
            
df['short_purpose'] = df['lemma_purpose'].apply(purpose_short) # создаем новый столбец с категориями

**Вывод**

Данные в таблице распредены по категориям, теперь можно приступить к анализу данных.

## Шаг 3. Ответьте на вопросы

*Чтобы понять зависимость, необходимо получить процентное соотношение клиентов с фактом непогашенного кредита к общему числу, распределенное по категориям.*

- Есть ли зависимость между наличием детей и возвратом кредита в срок?


In [30]:
# найдем значения с непогашеным кредитом по категории и разделим на общее количетсво в этой категории

(df[df['debt'] == 1]['children_cat'].value_counts() # количество с задолженностью по категориям
 / df['children_cat'].value_counts() # общее количество по категориям
 * 100).sort_values(ascending=False) # сортируем по убыванию

2          9.492481
1          9.165808
более 3    8.157895
0          7.543822
Name: children_cat, dtype: float64

**Вывод**

У заёмщиков с 1 и 2 детьми большее процентное отношение, чем у бездетных.

Бездетные заёмщики чаще возвращают кредит в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
# найдем значения с непогашеным кредитом по категории и разделим на общее количетсво в этой категории

(df[df['debt'] == 1]['family_status'].value_counts() # количество с задолженностью по категориям
 / df['family_status'].value_counts() # общее количество по категориям
 * 100).sort_values(ascending=False) # сортируем по убыванию

Не женат / не замужем    9.750890
гражданский брак         9.347145
женат / замужем          7.545182
в разводе                7.112971
вдовец / вдова           6.569343
Name: family_status, dtype: float64

**Вывод**

Исходя из полученных данных, клиенты, состоящие в гражданском браке или не имеющие жену/мужа чаще не выплачивают кредит в срок, а вдовы/вдовцы - реже.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
# найдем значения с непогашеным кредитом по категории и разделим на общее количетсво в этой категории

(df[df['debt'] == 1]['total_income_cat'].value_counts() # количество с задолженностью по категориям
 / df['total_income_cat'].value_counts() # общее количество по категориям
 * 100).sort_values(ascending=False) # сортируем по убыванию

средний          8.997669
ниже среднего    8.366348
выше среднего    8.203216
низкий           8.016779
высокий          6.991377
Name: total_income_cat, dtype: float64

**Вывод**

Клиенты сразу по четырем категориям с доходом от низкого к выше среднему чаще не выплачивают кредит в срок, чем клиенты с высоким доходом.

Все таки, уровень дохода влияет на выплату кредита в срок. 

- Как разные цели кредита влияют на его возврат в срок?

In [33]:
# найдем значения с непогашеным кредитом по категории и разделим на общее количетсво в этой категории

(df[df['debt'] == 1]['short_purpose'].value_counts() # количество с задолженностью по категориям
 / df['short_purpose'].value_counts() # общее количество по категориям
 * 100).sort_values(ascending=False) # сортируем по убыванию

автомобиль      9.359034
образование     9.220035
свадьба         8.003442
недвижимость    7.463392
жилье           6.905830
Name: short_purpose, dtype: float64

Проверим ранее утвержденную гипотезу и переименуем значения с жильем в недвижимость:

In [34]:
df['short_purpose'] = df['short_purpose'].replace('жилье', 'недвижимость')

(df[df['debt'] == 1]['short_purpose'].value_counts() # количество с задолженностью по категориям
 / df['short_purpose'].value_counts() # общее количество по категориям
 * 100).sort_values(ascending=False) # сортируем по убыванию

автомобиль      9.359034
образование     9.220035
свадьба         8.003442
недвижимость    7.233373
Name: short_purpose, dtype: float64

**Вывод**

Ранее выдвинутая гипотеза не сильно повлияла на исходный результат:

По операциям с автомобилями и образованием, клиенты чаще не выплачивают в срок.

По операциям с недвижимостью и жилем, наоборот, выплачивают чаще в срок.

- Есть ли зависимость между уровнем образования и возвратом кредита в срок?

In [35]:
# найдем значения с непогашеным кредитом по категории и разделим на общее количетсво в этой категории

(df[df['debt'] == 1]['education'].value_counts() # количество с задолженностью по категориям
 / df['education'].value_counts() # общее количество по категориям
 * 100).sort_values(ascending=False) # сортируем по убыванию

начальное              10.992908
неоконченное высшее     9.139785
среднее                 8.990245
высшее                  5.295238
ученая степень               NaN
Name: education, dtype: float64

**Вывод**

У людей с начальным образованием процентное соотношение выше 10, что ранее не было в значениях по другим категориям.

И с высшим образованием самый низкий процент.

*Что удивительно, у людей с ученой степенью нет задолженностей вовсе!* Но это лишь потому, что таких людей очень мало в выборке - всего 6.

## Шаг 4. Общий вывод

**Проверены несколько гипотез и установлено:**

* количество детей влияет на выплату кредита в срок, так бездетные заёмщики чаще возвращают кредит в срок.

* семейный статус так же влияет на выплату кредита в срок: неженатые и незамужние чаще задерживают выплаты по кредиту, а вдовы и вдовцы - реже.

* исходя из данных по уровню дохода - нет особых различий между категориями `низкий` и `выше среднего`, но с высокими доходами реже не выплачивают в срок.

* если рассматривать по цели выдачи кредита - по операциям с автомобилями и образованием - чаще не выплачивают в срок.

Но самый необычный результат получился по уровню образования:
Клиенты с начальным образованием чаще не выплачивают в срок, а клиенты с высшим имеют самый низкий процент.

*Возможно*, данные исследования могут быть не точными, так как значения не сильно различаются между собой и не превышают 10%.

Чтобы более детально рассмотреть данные, необходимо рассматривать совокупность нескольных данных по нескольким столбцам и их отношение к невыплаченным в срок кредитам, так данные могут более детально отобразить ситуацию по невыплатам в срок.

**Кредитный скоринг** может работать лучше, если рассматривать большую совокупность данных (по нескольким столбцам).